<a href="https://colab.research.google.com/github/Kaivan-Khazeni/Machine_Learning/blob/main/HW1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import numpy as np


In [31]:
labels_value = ['unacc','acc','good','vgood']
#label is where the values of each item is stored.  It is not binary,
#it is unacceptable, acceptable, good, very good.  This is different 
#then before
attributes = ['buying','maint','doors','persons','lug_boot','safety','label']

In [32]:
df = pd.read_csv ('Cars/train.csv',names=attributes)
df.head(5)
df_test = pd.read_csv ('Cars/test.csv',names=attributes)
df_test.head(5)

,buying,maint,doors,persons,lug_boot,safety,label
0,vhigh,high,5more,2,small,low,unacc
1,low,low,5more,2,small,med,unacc
2,low,vhigh,4,2,med,low,unacc
3,high,vhigh,3,4,med,med,unacc
4,vhigh,low,4,4,med,low,unacc


In [33]:
test_dictB = {'A':['K','A','I','K','A'],
         'B':['A','A','C','B','B'],
         'Y':['yes','no','so','maybe','so']}
test_dfB = pd.DataFrame(test_dictB)
test_dfB['A']

0    K
1    A
2    I
3    K
4    A
Name: A, dtype: object

In [34]:
attribute_values = {
    "buying": df['buying'].unique(),
    "maint": df['maint'].unique(),
    "doors": df['doors'].unique(),
    "persons": df['persons'].unique(),
    "lug_boot": df['lug_boot'].unique(),
    "safety": df['safety'].unique()
}

In [111]:
check_for_numerical(bank_df_train)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,services,married,secondary,no,0,yes,no,unknown,0,may,0,0,1,0,unknown,no
1,1,blue-collar,single,secondary,no,0,yes,yes,cellular,0,feb,1,0,1,0,unknown,no
2,1,technician,married,secondary,no,1,no,yes,cellular,1,aug,1,0,1,1,success,yes
3,1,admin.,married,tertiary,no,0,yes,no,cellular,0,jul,1,0,1,0,unknown,no
4,0,management,single,tertiary,no,1,no,no,cellular,0,apr,0,0,1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1,technician,divorced,secondary,no,0,yes,no,cellular,1,may,0,0,1,1,other,no
4996,1,management,divorced,tertiary,no,0,yes,no,telephone,1,oct,1,0,1,0,unknown,yes
4997,0,blue-collar,married,secondary,no,0,yes,no,unknown,0,jun,1,0,1,0,unknown,no
4998,0,management,single,tertiary,no,1,yes,no,unknown,0,may,0,0,1,0,unknown,no


eh


In [124]:
def check_for_numerical(df_S):
  import statistics
  columns_with_number = df_S.select_dtypes(include=np.number).columns.tolist()
  
  df_return = df_S
  for attr in columns_with_number:
    median = statistics.median(df_S[attr])
    df_return.loc[(df_S[attr] <= median),attr] = 0.0
    df_return.loc[(df_S[attr] > median),attr] = 1.0
  return df_return


    

In [36]:
def find_best_split(max_labels,variant,attr,S,total_ent):
  #Need to check the
  
  list_of_IG = {}
  for A1 in attr:
    curr_attr = A1
    curr_df = S[[curr_attr,S.keys()[-1]]]
    
    len_A1_df = len(curr_df[curr_attr])
    found_sum_expected_entropy = 0
    all_attr_values = curr_df[curr_attr].unique()
    for A2 in all_attr_values:
      
      attribute_specific = curr_df[curr_df[curr_attr] == A2]
      found_expected_entropy = 0
      entropy = find_total_entropy(variant,attribute_specific)
      if (entropy == 1 and variant == 2):
        entropy = 0
      found_sum_expected_entropy += (entropy * (len(attribute_specific)/len_A1_df))
    list_of_IG[A1] = total_ent - found_sum_expected_entropy   
  return list_of_IG


In [38]:
def find_total_entropy(max_labels,variant,S):
  if variant == 1:
    #use entropy
    type_name = S.keys()[-1]
    entropy = 0
    label_names = S[type_name].unique()
    for val in label_names:
      #print(val)
      p = S[type_name].value_counts()[val] / len(S[type_name])
      #print(p)
      entropy += -p*np.log2(p)
    return entropy
  elif variant == 2:
    counter = len(labels_value)
    
    #use ME
    type_name = S.keys()[-1]  
    #ME = 0
    label_names = S[type_name].unique()
    list_of_prob = []
    for val in label_names:
      p = S[type_name].value_counts()[val] / len(S)
      list_of_prob.append(p)
  
    if len(list_of_prob) == max_labels:
      return np.min(list_of_prob)
    else:
      return 0
  elif variant == 3:
    #use gini
    type_name = S.keys()[-1]
    #Gini = 0
    label_names = S[type_name].unique()
    sum_p = 0
    for val in label_names:
      p = S[type_name].value_counts()[val] / len(S[type_name])
      sum_p += np.square(p)

    return 1 - sum_p

In [280]:
#max depth will range from 1 - 6
def ID3(max_labels, max_depth,variant,S,A,tree=None):
  _S = S
  if A == "none":
    _S = check_for_numerical(S)
    #check for unknown here:
    for attr in _S.columns:
      if "unknown" in _S[attr].unique():
        if _S[attr].value_counts().keys()[0] == "unknown":
          _S.loc[(_S[attr] == "unknown"),attr] = _S[attr].value_counts().keys()[1]
        else:
          _S.loc[(_S[attr] == "unknown"),attr] = _S[attr].value_counts().keys()[0]

  S = _S

  #if len(S[S.keys()[-1]].unique()) > max_labels:
  max_labels = len(S[S.keys()[-1]].unique())
  labels_value = S[S.keys()[-1]].unique()
  Class = S.keys()[-1]
  #Step 1: check if all examplesa have the same label
  #keep counter to make sure iterations do not exceed the max depth
  if max_depth >= 0:
    
    #Finding root based off of gain
    #A. Find Total entropy using variant
    total_entropy = find_total_entropy(max_labels, variant,S)
    #B. Now find the best split, which is A.
    table_attributes = S.loc[:,S.columns != S.keys()[-1]].columns


    best_A = find_best_split(max_labels,variant,table_attributes,S,total_entropy)

    
    A = sorted(best_A.items(), key=lambda x: x[1],reverse=True)[0][0]
    
    if tree is None:                    
      tree={}
      tree[A] = {}

   
    try:
      values_of_A = S[A].unique()
    except:
      print(S)

    for A_val in values_of_A:
      Sv = S.loc[S[A]==A_val,S.columns != A]
      common,counts = np.unique(Sv[Sv.keys()[-1]],return_counts=True)
      if len(Sv.columns) == 1 or len(counts) == 1 or max_depth == 0:
        tree[A][A_val] = common[0]
      else:
        tree[A][A_val] = ID3(max_labels,max_depth-1,variant,Sv,A,tree=None)     
    return tree
    



In [234]:
# https://stackoverflow.com/questions/34836777/print-complete-key-path-for-all-the-values-of-a-python-nested-dictionary
def dict_path(my_dict, path=None):
    if path is None:
        path = []
    for k,v in my_dict.items():
        newpath = path + [k]
        if isinstance(v, dict):
            for u in dict_path(v, newpath):
                yield u
        else:
            yield newpath, v

In [196]:
#TRAINING ERROR
# DEPTH 1 -> 6
# RECORD ERROR
list_of_variants = ["Entropy","MajorityError","Gini Index"]
error_dict_per_variant = {}

testing_error = {}
for b in range(1,4):
  error_dict_per_variant[list_of_variants[b-1]] = {}
  total = 0
  total_test = 0
  testing_error[list_of_variants[b-1]] = {}
  for i in range(1,7):
    
    error_counter = 0
    error_counter_test = 0

    train_tree = ID3(-1*float("inf"),i,b,df,"none",tree=None)
    train_tree_path = list(dict_path(train_tree,path=None))


    #print(train_tree_path)
    #df_filter = df
    #df_filter_test = df_test
    for j in range(len(train_tree_path)):
      df_filter = df
      df_filter_test = df_test
      curr_path = train_tree_path[j]
      for a in range(0,len(curr_path[0])):
        if (a % 2) == 0:
          #print(curr_path)
          #print(curr_path[0])
          df_filter = df_filter[df_filter[curr_path[0][a]] == curr_path[0][a+1]]
          df_filter_test = df_filter_test[df_filter_test[curr_path[0][a]] == curr_path[0][a+1]]
      

      length = len(df_filter)
      length_test = len(df_filter_test)

      count_of_accuracies = len(df_filter[df_filter[df_filter.keys()[-1]] == curr_path[1]]) 
      count_of_accuracies_test = len(df_filter_test[df_filter_test[df_filter_test.keys()[-1]] == curr_path[1]]) 

      count_of_inaccuracies = length - count_of_accuracies
      count_of_inaccuracies_test = length_test - count_of_accuracies_test

      
      if length == 0:
        error_counter += 0
      else:
        error_counter += count_of_inaccuracies/length
      if length_test == 0:
        error_counter_test += 0
      else:
        error_counter_test += count_of_inaccuracies_test/length_test

    total += error_counter/len(train_tree_path)
    total_test += error_counter_test / len(train_tree_path)

  
  error_dict_per_variant[list_of_variants[b-1]] = total / 6    
  testing_error[list_of_variants[b-1]] = total_test / 6    




In [171]:
error_dict_per_variant
testing_error

{'Entropy': 0.1652520710570246,
 'Gini Index': 0.1652520710570246,
 'MajorityError': 0.2735988698705201}

In [197]:
Errors = {'Train Average Error':[error_dict_per_variant['Entropy'],error_dict_per_variant['MajorityError'],error_dict_per_variant['Gini Index']],
         'Test Average Error':[testing_error['Entropy'],testing_error['MajorityError'],testing_error['Gini Index']]}
         
error_df = pd.DataFrame(Errors, index=['Entropy','MajorityError','Gini Index'])
error_df

,Train Average Error,Test Average Error
Entropy,0.173247,0.226531
MajorityError,0.266058,0.330606
Gini Index,0.172259,0.225682


In [48]:
bank_attributes = ["age","job","marital","education","default","balance","housing","loan","contact","day","month","duration","campaign","pdays","previous","poutcome","y"]

In [285]:
bank_df_train = pd.read_csv ('Bank/train.csv',names=bank_attributes)
bank_df_train.head(5)
bank_df_test = pd.read_csv ('Bank/test.csv',names=bank_attributes)
bank_df_test.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,41,management,single,secondary,no,764,no,no,cellular,12,jun,230,2,-1,0,unknown,no
1,39,blue-collar,married,secondary,no,49,yes,no,cellular,14,may,566,1,370,2,failure,no
2,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown,no
3,31,entrepreneur,single,tertiary,no,247,yes,yes,unknown,2,jun,273,1,-1,0,unknown,no
4,26,student,single,unknown,no,2020,no,no,telephone,28,jan,42,3,-1,0,unknown,no


In [289]:
#TRAINING ERROR
# DEPTH 1 -> 17
# RECORD ERROR
list_of_variants = ["Entropy","MajorityError","Gini Index"]
error_dict_per_variant = {}

testing_error = {}
for b in range(1,4):
  error_dict_per_variant[list_of_variants[b-1]] = {}
  total = 0
  total_test = 0
  testing_error[list_of_variants[b-1]] = {}
  for i in range(1,17):
    
    error_counter = 0
    error_counter_test = 0

  
    train_tree = ID3(-1*float("inf"),i,b,bank_df_train,"none",tree=None)
    train_tree_path = list(dict_path(train_tree,path=None))


    for j in range(len(train_tree_path)):
      df_filter = bank_df_train
      df_filter_test = bank_df_test
      curr_path = train_tree_path[j]
      for a in range(0,len(curr_path[0])):
        if (a % 2) == 0:
          df_filter = df_filter[df_filter[curr_path[0][a]] == curr_path[0][a+1]]
          df_filter_test = df_filter_test[df_filter_test[curr_path[0][a]] == curr_path[0][a+1]]
      

      length = len(df_filter)
      length_test = len(df_filter_test)

      count_of_accuracies = len(df_filter[df_filter[df_filter.keys()[-1]] == curr_path[1]]) 
      count_of_accuracies_test = len(df_filter_test[df_filter_test[df_filter_test.keys()[-1]] == curr_path[1]]) 

      count_of_inaccuracies = length - count_of_accuracies
      count_of_inaccuracies_test = length_test - count_of_accuracies_test

      
      if length == 0:
        error_counter += 0
      else:
        error_counter += count_of_inaccuracies/length
      if length_test == 0:
        error_counter_test += 0
      else:
        error_counter_test += count_of_inaccuracies_test/length_test

    total += error_counter/len(train_tree_path)
    total_test += error_counter_test / len(train_tree_path)

  
  error_dict_per_variant[list_of_variants[b-1]] = total / 16    
  testing_error[list_of_variants[b-1]] = total_test / 16    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packa

In [287]:
train_tree = ID3(-1*float("inf"),16,1,bank_df_train,"none",tree=None)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


{'duration': {0: {'month': {'apr': {'job': {'admin.': {'education': {'secondary': 'no',
        'tertiary': {'housing': {'no': 'yes', 'yes': 'no'}}}},
      'blue-collar': {'poutcome': {'failure': 'no',
        'other': 'no',
        'success': 'yes'}},
      'housemaid': 'no',
      'management': {'housing': {'no': {'balance': {0: 'no',
          1: {'age': {0: {'campaign': {0: 'yes', 1: 'no'}}, 1: 'no'}}}},
        'yes': 'no'}},
      'retired': 'no',
      'self-employed': {'marital': {'divorced': 'yes',
        'married': 'no',
        'single': 'no'}},
      'services': 'no',
      'student': {'balance': {0: 'yes', 1: 'no'}},
      'technician': {'day': {0: {'marital': {'divorced': 'yes',
          'married': {'balance': {0: 'yes',
            1: {'housing': {'no': 'yes', 'yes': 'no'}}}},
          'single': 'no'}},
        1: 'no'}},
      'unemployed': 'no'}},
    'aug': {'previous': {0: {'job': {'admin.': 'no',
        'blue-collar': 'no',
        'entrepreneur': 'no',
       

In [290]:
Errors_bank = {'Train Average Error':[error_dict_per_variant['Entropy'],error_dict_per_variant['MajorityError'],error_dict_per_variant['Gini Index']],
         'Test Average Error':[testing_error['Entropy'],testing_error['MajorityError'],testing_error['Gini Index']]}
         
error_df_bank = pd.DataFrame(Errors_bank, index=['Entropy','MajorityError','Gini Index'])
error_df_bank

,Train Average Error,Test Average Error
Entropy,0.056644,0.001545
MajorityError,0.058197,0.040242
Gini Index,0.062800,0.036321
